In [1]:
import pandas as pd
import os
import numpy as np
import scipy as sp
import torch

In [2]:
from scipy.stats import uniform, norm
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt

In [3]:
A = 1.2
b = -0.3
sigma = 0.2
N = 100

In [4]:
X = torch.from_numpy(uniform.rvs(size=N))
X.detach()
X = X.float()

In [5]:
e = torch.from_numpy(norm.rvs(loc=0, scale=sigma, size=N)).float()

In [6]:
y = A*X + b + e

In [7]:
y.detach()
y = y.float()

In [8]:
def sigma_to_rho(sigma):
    rho = torch.log(torch.exp(sigma) - 1)
    return rho

In [9]:
def rho_to_sigma(rho):
    sigma = torch.log(1 + torch.exp(rho))
    return sigma

In [10]:
def log_q_pdf(w, theta):
    (mu_A, mu_b, rho_A, rho_b) = theta 
    
    sigma_A = rho_to_sigma(rho_A)
    qA = torch.distributions.normal.Normal(mu_A, sigma_A)
    
    sigma_b = rho_to_sigma(rho_b)
    qb = torch.distributions.normal.Normal(mu_b, sigma_b)
    
    lp = qA.log_prob(w[0]) + qb.log_prob(w[1])
    return lp

In [11]:
def log_p_pdf(w):
    prior_mu_A = torch.tensor(0.0) 
    prior_sigma_A = torch.tensor(1.0)
    pA = torch.distributions.normal.Normal(prior_mu_A, prior_sigma_A)
    
    prior_mu_b = torch.tensor(0.0) 
    prior_sigma_b = torch.tensor(1.0)
    pb = torch.distributions.normal.Normal(prior_mu_b, prior_sigma_b)
    
    lp = pA.log_prob(w[0]) + pb.log_prob(w[1])
    return lp

In [12]:
def likelihood(y,X,w):
    A = w[0]
    b = w[1]
    
    y_pred = A*X + b
    lp = torch.distributions.normal.Normal(y_pred, 0.2).log_prob(y).sum()
    return lp

In [13]:
def f(w, theta):
    v = log_q_pdf(w, theta) - log_p_pdf(w) - likelihood(y,X,w)
    return v

In [14]:
mu_A = torch.tensor(0.0)
sigma_A = torch.tensor(0.1)
rho_A = sigma_to_rho(sigma_A)
mu_b = torch.tensor(0.0)
sigma_b = torch.tensor(0.1)
rho_b = sigma_to_rho(sigma_b)

theta = torch.tensor([mu_A, mu_b, rho_A, rho_b], requires_grad=True)

In [15]:

lst_grad_mu = []
lst_grad_rho = []
for j in range(0,5000):
    epsilon = torch.distributions.normal.Normal(torch.tensor(0.0) , torch.tensor(1.0)).sample(torch.Size([2,]))

    with torch.no_grad():
        w = theta[0:2].clone().detach() + torch.log(1 + torch.exp(theta[2:4].clone().detach()))*epsilon
    w.requires_grad = True

    v = f(w, theta)   
    v.backward()
    v = torch.tensor([])

    with torch.no_grad():
        grad_mu = w.grad + theta.grad[0:2]
        grad_rho = w.grad*epsilon/(1 + torch.exp(-theta[2:4])) + -theta.grad[2:4]


    lst_grad_mu.append(grad_mu)
    lst_grad_rho.append(grad_rho)

    theta.grad.zero_()
    w.grad.zero_()
    v.detach()
grad_mu = torch.stack(lst_grad_mu).mean(dim=0)
grad_rho = torch.stack(lst_grad_rho).mean(dim=0)
        
 



In [16]:
grad_mu

tensor([-724.3985, -863.2999])

In [17]:
grad_rho

tensor([ 6.9006, 22.5250])

In [18]:
alpha = torch.tensor(10e-5)

In [19]:
for i in range(0,10000):
    lst_grad_mu = []
    lst_grad_rho = []
    for j in range(0,10):
        epsilon = torch.distributions.normal.Normal(torch.tensor(0.0) , torch.tensor(1.0)).sample(torch.Size([2,]))

        with torch.no_grad():
            w = theta[0:2].clone().detach() + torch.log(1 + torch.exp(theta[2:4].clone().detach()))*epsilon
        w.requires_grad = True

        v = f(w, theta)   
        v.backward()
        v = torch.tensor([])

        with torch.no_grad():
            grad_mu = w.grad + theta.grad[0:2]
            grad_rho = w.grad*epsilon/(1 + torch.exp(-theta[2:4])) + -theta.grad[2:4]


        lst_grad_mu.append(grad_mu)
        lst_grad_rho.append(grad_rho)

        theta.grad.zero_()
        w.grad.zero_()
        v.detach()
    grad_mu = torch.stack(lst_grad_mu).mean(dim=0)
    grad_rho = torch.stack(lst_grad_rho).mean(dim=0)
    
    #print(grad_mu)
    #print(grad_rho)
        
    with torch.no_grad():
        theta[0:2] -= alpha*grad_mu
        theta[2:4] -= alpha*grad_rho
    
    print(theta)



tensor([ 0.0751,  0.0924, -2.2519, -2.2588], requires_grad=True)
tensor([ 0.1250,  0.1379, -2.2537, -2.2600], requires_grad=True)
tensor([ 0.1666,  0.1711, -2.2534, -2.2613], requires_grad=True)
tensor([ 0.1999,  0.1885, -2.2540, -2.2631], requires_grad=True)
tensor([ 0.2330,  0.2100, -2.2526, -2.2660], requires_grad=True)
tensor([ 0.2591,  0.2170, -2.2524, -2.2705], requires_grad=True)
tensor([ 0.2815,  0.2162, -2.2536, -2.2731], requires_grad=True)
tensor([ 0.2981,  0.2067, -2.2535, -2.2749], requires_grad=True)
tensor([ 0.3182,  0.2056, -2.2541, -2.2773], requires_grad=True)
tensor([ 0.3310,  0.1914, -2.2546, -2.2798], requires_grad=True)
tensor([ 0.3452,  0.1801, -2.2550, -2.2818], requires_grad=True)
tensor([ 0.3635,  0.1773, -2.2551, -2.2828], requires_grad=True)
tensor([ 0.3740,  0.1606, -2.2565, -2.2853], requires_grad=True)
tensor([ 0.4006,  0.1726, -2.2587, -2.2868], requires_grad=True)
tensor([ 0.4155,  0.1648, -2.2594, -2.2878], requires_grad=True)
tensor([ 0.4360,  0.1670,

tensor([ 1.1135, -0.2207, -2.3387, -2.4876], requires_grad=True)
tensor([ 1.1194, -0.2139, -2.3402, -2.4903], requires_grad=True)
tensor([ 1.1185, -0.2201, -2.3405, -2.4923], requires_grad=True)
tensor([ 1.1199, -0.2224, -2.3411, -2.4933], requires_grad=True)
tensor([ 1.1201, -0.2267, -2.3416, -2.4944], requires_grad=True)
tensor([ 1.1178, -0.2327, -2.3423, -2.4953], requires_grad=True)
tensor([ 1.1224, -0.2284, -2.3427, -2.4956], requires_grad=True)
tensor([ 1.1277, -0.2234, -2.3436, -2.4970], requires_grad=True)
tensor([ 1.1329, -0.2171, -2.3440, -2.4982], requires_grad=True)
tensor([ 1.1325, -0.2213, -2.3447, -2.5002], requires_grad=True)
tensor([ 1.1315, -0.2259, -2.3452, -2.5017], requires_grad=True)
tensor([ 1.1287, -0.2332, -2.3456, -2.5029], requires_grad=True)
tensor([ 1.1311, -0.2330, -2.3465, -2.5053], requires_grad=True)
tensor([ 1.1297, -0.2384, -2.3468, -2.5057], requires_grad=True)
tensor([ 1.1304, -0.2411, -2.3473, -2.5061], requires_grad=True)
tensor([ 1.1383, -0.2310,

tensor([ 1.2003, -0.2636, -2.4224, -2.6451], requires_grad=True)
tensor([ 1.1950, -0.2747, -2.4230, -2.6474], requires_grad=True)
tensor([ 1.2011, -0.2654, -2.4243, -2.6491], requires_grad=True)
tensor([ 1.1969, -0.2718, -2.4255, -2.6504], requires_grad=True)
tensor([ 1.1934, -0.2773, -2.4259, -2.6517], requires_grad=True)
tensor([ 1.1899, -0.2830, -2.4265, -2.6523], requires_grad=True)
tensor([ 1.1933, -0.2767, -2.4263, -2.6529], requires_grad=True)
tensor([ 1.1941, -0.2766, -2.4266, -2.6536], requires_grad=True)
tensor([ 1.1857, -0.2914, -2.4273, -2.6547], requires_grad=True)
tensor([ 1.1920, -0.2792, -2.4279, -2.6553], requires_grad=True)
tensor([ 1.1920, -0.2806, -2.4282, -2.6563], requires_grad=True)
tensor([ 1.1974, -0.2720, -2.4293, -2.6572], requires_grad=True)
tensor([ 1.1936, -0.2785, -2.4300, -2.6583], requires_grad=True)
tensor([ 1.1931, -0.2805, -2.4302, -2.6589], requires_grad=True)
tensor([ 1.2014, -0.2671, -2.4307, -2.6595], requires_grad=True)
tensor([ 1.1976, -0.2736,

tensor([ 1.2008, -0.2747, -2.4878, -2.7690], requires_grad=True)
tensor([ 1.2035, -0.2695, -2.4882, -2.7700], requires_grad=True)
tensor([ 1.2035, -0.2698, -2.4885, -2.7706], requires_grad=True)
tensor([ 1.2057, -0.2664, -2.4890, -2.7714], requires_grad=True)
tensor([ 1.2098, -0.2604, -2.4891, -2.7718], requires_grad=True)
tensor([ 1.2072, -0.2657, -2.4893, -2.7725], requires_grad=True)
tensor([ 1.2038, -0.2719, -2.4896, -2.7730], requires_grad=True)
tensor([ 1.2084, -0.2639, -2.4901, -2.7735], requires_grad=True)
tensor([ 1.2046, -0.2717, -2.4904, -2.7751], requires_grad=True)
tensor([ 1.2011, -0.2760, -2.4911, -2.7760], requires_grad=True)
tensor([ 1.2010, -0.2755, -2.4915, -2.7763], requires_grad=True)
tensor([ 1.2020, -0.2741, -2.4922, -2.7780], requires_grad=True)
tensor([ 1.2084, -0.2620, -2.4933, -2.7797], requires_grad=True)
tensor([ 1.2064, -0.2649, -2.4940, -2.7812], requires_grad=True)
tensor([ 1.2022, -0.2721, -2.4948, -2.7824], requires_grad=True)
tensor([ 1.2057, -0.2655,

tensor([ 1.2009, -0.2735, -2.5406, -2.8594], requires_grad=True)
tensor([ 1.2023, -0.2706, -2.5410, -2.8601], requires_grad=True)
tensor([ 1.2021, -0.2704, -2.5411, -2.8606], requires_grad=True)
tensor([ 1.2045, -0.2654, -2.5418, -2.8619], requires_grad=True)
tensor([ 1.2054, -0.2633, -2.5418, -2.8630], requires_grad=True)
tensor([ 1.2096, -0.2548, -2.5418, -2.8635], requires_grad=True)
tensor([ 1.2092, -0.2565, -2.5421, -2.8639], requires_grad=True)
tensor([ 1.2123, -0.2508, -2.5425, -2.8647], requires_grad=True)
tensor([ 1.2118, -0.2496, -2.5425, -2.8650], requires_grad=True)
tensor([ 1.2068, -0.2595, -2.5428, -2.8656], requires_grad=True)
tensor([ 1.1990, -0.2713, -2.5436, -2.8666], requires_grad=True)
tensor([ 1.1979, -0.2722, -2.5439, -2.8676], requires_grad=True)
tensor([ 1.1954, -0.2768, -2.5442, -2.8683], requires_grad=True)
tensor([ 1.2000, -0.2677, -2.5446, -2.8694], requires_grad=True)
tensor([ 1.1976, -0.2718, -2.5449, -2.8703], requires_grad=True)
tensor([ 1.1939, -0.2785,

...

tensor([ 1.2014, -0.2692, -3.2966, -3.8140], requires_grad=True)
tensor([ 1.2014, -0.2691, -3.2967, -3.8140], requires_grad=True)
tensor([ 1.1999, -0.2720, -3.2967, -3.8139], requires_grad=True)
tensor([ 1.2020, -0.2686, -3.2967, -3.8139], requires_grad=True)
tensor([ 1.2009, -0.2702, -3.2968, -3.8140], requires_grad=True)
tensor([ 1.2011, -0.2689, -3.2967, -3.8139], requires_grad=True)
tensor([ 1.2020, -0.2680, -3.2967, -3.8139], requires_grad=True)
tensor([ 1.2014, -0.2685, -3.2967, -3.8140], requires_grad=True)
tensor([ 1.1995, -0.2722, -3.2967, -3.8140], requires_grad=True)
tensor([ 1.2010, -0.2697, -3.2968, -3.8140], requires_grad=True)
tensor([ 1.1999, -0.2716, -3.2968, -3.8141], requires_grad=True)
tensor([ 1.1973, -0.2758, -3.2968, -3.8141], requires_grad=True)
tensor([ 1.1964, -0.2770, -3.2968, -3.8141], requires_grad=True)
tensor([ 1.1974, -0.2753, -3.2969, -3.8142], requires_grad=True)
tensor([ 1.1971, -0.2754, -3.2969, -3.8141], requires_grad=True)
tensor([ 1.1978, -0.2739,

tensor([ 1.2011, -0.2703, -3.3002, -3.8172], requires_grad=True)
tensor([ 1.2008, -0.2709, -3.3002, -3.8172], requires_grad=True)
tensor([ 1.1997, -0.2731, -3.3003, -3.8172], requires_grad=True)
tensor([ 1.2006, -0.2721, -3.3003, -3.8172], requires_grad=True)
tensor([ 1.2016, -0.2702, -3.3003, -3.8172], requires_grad=True)
tensor([ 1.2016, -0.2701, -3.3003, -3.8172], requires_grad=True)
tensor([ 1.2038, -0.2666, -3.3003, -3.8173], requires_grad=True)
tensor([ 1.2034, -0.2675, -3.3003, -3.8174], requires_grad=True)
tensor([ 1.2019, -0.2701, -3.3004, -3.8174], requires_grad=True)
tensor([ 1.2015, -0.2711, -3.3005, -3.8174], requires_grad=True)
tensor([ 1.2017, -0.2712, -3.3004, -3.8174], requires_grad=True)
tensor([ 1.2018, -0.2710, -3.3005, -3.8174], requires_grad=True)
tensor([ 1.2033, -0.2682, -3.3005, -3.8173], requires_grad=True)
tensor([ 1.2015, -0.2713, -3.3005, -3.8173], requires_grad=True)
tensor([ 1.2038, -0.2680, -3.3005, -3.8173], requires_grad=True)
tensor([ 1.2025, -0.2697,

tensor([ 1.1993, -0.2740, -3.3026, -3.8193], requires_grad=True)
tensor([ 1.2006, -0.2720, -3.3025, -3.8193], requires_grad=True)
tensor([ 1.2002, -0.2720, -3.3026, -3.8192], requires_grad=True)
tensor([ 1.2002, -0.2718, -3.3025, -3.8193], requires_grad=True)
tensor([ 1.1995, -0.2730, -3.3026, -3.8194], requires_grad=True)
tensor([ 1.2013, -0.2699, -3.3026, -3.8194], requires_grad=True)
tensor([ 1.1994, -0.2727, -3.3025, -3.8193], requires_grad=True)
tensor([ 1.1997, -0.2720, -3.3025, -3.8194], requires_grad=True)
tensor([ 1.2004, -0.2709, -3.3025, -3.8194], requires_grad=True)
tensor([ 1.1996, -0.2720, -3.3025, -3.8195], requires_grad=True)
tensor([ 1.2000, -0.2705, -3.3025, -3.8194], requires_grad=True)
tensor([ 1.1994, -0.2713, -3.3025, -3.8195], requires_grad=True)
tensor([ 1.2017, -0.2671, -3.3024, -3.8194], requires_grad=True)
tensor([ 1.2018, -0.2673, -3.3025, -3.8194], requires_grad=True)
tensor([ 1.2020, -0.2663, -3.3025, -3.8194], requires_grad=True)
tensor([ 1.2023, -0.2657,

tensor([ 1.1967, -0.2751, -3.3041, -3.8212], requires_grad=True)
tensor([ 1.1972, -0.2744, -3.3042, -3.8212], requires_grad=True)
tensor([ 1.1978, -0.2735, -3.3042, -3.8212], requires_grad=True)
tensor([ 1.1949, -0.2778, -3.3042, -3.8212], requires_grad=True)
tensor([ 1.1946, -0.2785, -3.3043, -3.8212], requires_grad=True)
tensor([ 1.1987, -0.2715, -3.3043, -3.8213], requires_grad=True)
tensor([ 1.1987, -0.2712, -3.3043, -3.8214], requires_grad=True)
tensor([ 1.1983, -0.2719, -3.3043, -3.8213], requires_grad=True)
tensor([ 1.2004, -0.2688, -3.3044, -3.8212], requires_grad=True)
tensor([ 1.2004, -0.2692, -3.3044, -3.8212], requires_grad=True)
tensor([ 1.2003, -0.2692, -3.3045, -3.8213], requires_grad=True)
tensor([ 1.2013, -0.2676, -3.3044, -3.8213], requires_grad=True)
tensor([ 1.1997, -0.2706, -3.3045, -3.8214], requires_grad=True)
tensor([ 1.2005, -0.2686, -3.3045, -3.8214], requires_grad=True)
tensor([ 1.2005, -0.2691, -3.3045, -3.8215], requires_grad=True)
tensor([ 1.2018, -0.2675,

tensor([ 1.2015, -0.2675, -3.3068, -3.8232], requires_grad=True)
tensor([ 1.1991, -0.2717, -3.3069, -3.8231], requires_grad=True)
tensor([ 1.2003, -0.2697, -3.3069, -3.8231], requires_grad=True)
tensor([ 1.1994, -0.2719, -3.3070, -3.8232], requires_grad=True)
tensor([ 1.1992, -0.2723, -3.3070, -3.8231], requires_grad=True)
tensor([ 1.1998, -0.2710, -3.3070, -3.8232], requires_grad=True)
tensor([ 1.2004, -0.2696, -3.3070, -3.8232], requires_grad=True)
tensor([ 1.2003, -0.2695, -3.3070, -3.8232], requires_grad=True)
tensor([ 1.2011, -0.2681, -3.3071, -3.8232], requires_grad=True)
tensor([ 1.2027, -0.2660, -3.3071, -3.8232], requires_grad=True)
tensor([ 1.2014, -0.2681, -3.3071, -3.8232], requires_grad=True)
tensor([ 1.2007, -0.2695, -3.3071, -3.8232], requires_grad=True)
tensor([ 1.2023, -0.2672, -3.3072, -3.8233], requires_grad=True)
tensor([ 1.2001, -0.2712, -3.3073, -3.8234], requires_grad=True)
tensor([ 1.2006, -0.2704, -3.3073, -3.8234], requires_grad=True)
tensor([ 1.2023, -0.2675,

In [20]:
rho_to_sigma(torch.tensor(0.5488))

tensor(1.0047)

In [21]:
rho_to_sigma(torch.tensor(-3.2))

tensor(0.0486)

In [22]:
0.0486*0.0486

0.0023619599999999998